In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import pathlib
import pickle
from pathlib import Path
sys.path.append(f"{Path.home()}/morpho_repo")
sys.path.append(f"{Path.home()}/morpho_repo/pytorch")

sys.path.append(f"{Path.home()}/morpho_repo")
sys.path.append(f"{Path.home()}/morpho_repo/turing_codebase")

import pandas as pd

In [2]:
df = pd.read_csv("../Three_nodes_models/circuit_3954/df_network_analysis_full_topology.csv")

In [81]:
def simulation_run(args):
    import sys
    import pathlib
    import pickle
    from pathlib import Path
    sys.path.append(f"{Path.home()}/morpho_repo")
    sys.path.append(f"{Path.home()}/morpho_repo/turing_codebase")

    from turing.three_nodes_circuits import create_circuit_3954
    from pde_solvers.cn import RD_2D_1st_Order, Neumann_Boundary_2D
    from intro import find_roots, init_values
    
    run_ids, params_list, D_A, D_B = args
    model = RD_2D_1st_Order(Ds=[D_A, D_B, 0], 
                               delta_t=100/5000, 
                               Lx=10, Ly=10, 
                               Ix=128, Jy=128,
                               boundary_condition=Neumann_Boundary_2D)
    def to(arr):
        return arr.reshape(model.Ix, model.Jy)
    for run_id, params in zip(run_ids, params_list):
        kinetics = create_circuit_3954(params["n"],params["b_A"], params["mu_A"], params["V_A"], 
                                       params["K_AA"], params["K_AB"], params["K_AC"],
                                       params["b_B"], params["mu_B"], params["V_B"], 
                                       params["K_BA"], params["K_BC"],
                                       params["b_C"], params["mu_C"], params["V_C"], 
                                       params["K_CB"], params["K_CC"])
        
        roots = find_roots(kinetics)
        # Parameters with no fixed point are not
        # considrered
        if len(roots) == 0:
            continue        
        (A_init, B_init, C_init) = init_values(model, *roots[0])
        solution1 = model.integrate([A_init, B_init, C_init],kinetics, 5000, record_steps=50)
        # restrat for a longer run, but with saving in lower-resolution time 
        solution2 = model.integrate([solution1[-1, 0, :], 
                                     solution1[-1, 1, :],
                                     solution1[-1, 2, :]],kinetics, 200*5000, record_steps=5000)
        #################################################        
        run_params = {'Lx':model.Lx,
                      'Ly':model.Ly,
                      'Ix':model.Ix,
                      'Jy':model.Jy,
                      'delta_t':model.Δt,
                      'init_snpashots_delta_t': 1,
                      'init_solution_T_steps': 100,
                      'late_snpashots_delta_t': 100,
                      'late_solution_T_steps': 200,
                      'late_snpashots_index': 101
                     }
        solution_saved = np.vstack([np.stack([to(A_init), to(B_init), to(C_init)])[np.newaxis, ...], 
                                    solution1.reshape((-1, 3, 128, 128)),
                                    solution2.reshape((-1, 3, 128, 128))])
        
        params = {"D_A":D_A, "D_B":D_B,
                  **params}
        with open(f"./inputs/solution_{run_id}.pkl", "wb") as f:
            pickle.dump((run_params, params, solution_saved), f)
            
        del solution1
        del solution2
        del solution_saved
    del model
    
    

In [4]:
def row_to_param(param_string_to_dict):
    (n_val, 
     b_A_val, mu_A_val, V_A_val, K_AA_val, K_AB_val, K_AC_val,
     b_B_val, mu_B_val, V_B_val, K_BA_val, K_BC_val,
     b_C_val, mu_C_val, V_C_val, K_CB_val, K_CC_val) = eval(param_string_to_dict.replace(' ', ','))
    return {'n': n_val,
          'b_A': b_A_val, 'mu_A': mu_A_val, 'V_A': V_A_val,
          'K_AA': K_AA_val, 'K_AB': K_AB_val, 'K_AC': K_AC_val,
          'b_B': b_B_val, 'mu_B': mu_B_val, 'V_B': V_B_val,
          'K_BA': K_BA_val, 'K_BC': K_BC_val,
          'b_C': b_C_val, 'mu_C': mu_C_val, 'V_C': V_C_val,
          'K_CB': K_CB_val, 'K_CC': K_CC_val}    

In [5]:
df_subset = df[(df['type'] != 'wrong') & (df['type'] != 'odd')]

In [79]:
D_A, D_B = 0.01, 1.0
params_dict = [ row_to_param(row) for i, row in enumerate(df_subset["params"])] 
ids = [ i+1 for i, row in enumerate(df_subset["params"])] 
run_per_proc = 3

run_params = [ (ids[i:i+run_per_proc],
                params_dict[i:i+run_per_proc],
                D_A, D_B) 
               for i in range(0,len(params_dict), run_per_proc)]
    

In [80]:
len(ids), len(params_dict), len(run_params), len(run_params)*run_per_proc

(86, 86, 29, 87)

In [82]:
from multiprocessing import Pool

with Pool(12) as pool:
    res = pool.map(simulation_run, run_params[:12])

/end/home/rh2310/.conda/envs/pytorch/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:412: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)
/end/home/rh2310/.conda/envs/pytorch/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:412: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)
/end/home/rh2310/.conda/envs/pytorch/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:412: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)
/end/home/rh2310/.conda/envs/pytorch/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:412: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)
/end